# isic_dataloading

> How to load isic datasets for supervised learning, and possibly later SSL

Important: Note that have to download the data first from https://drive.google.com/drive/folders/1i6fsvK3ASO-AcMHmlQGYWWeM-rK9E_Rz?usp=drive_link. 

 

In [ ]:
#| default_exp isic_dataloading

In [ ]:
#| export

#directory = "/content/drive/MyDrive/ISIC_2019_Training_Input/"

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import torch
from fastai.vision.all import *
# from self_supervised.augmentations import *
# from self_supervised.layers import *
from base_rbt.utils import *
import re
import pandas as pd
import os
from collections import Counter

import re
import csv

import shutil
import zipfile

In [ ]:
#| export 

def unzip_and_cleanup(zip_file_path, target_dir):
    """
    Unzips the dataset files and cleans up the directory structure by removing a hardcoded unwanted prefix.
    
    Parameters:
    - zip_file_path: Full path to the zip file.
    - target_dir: Directory where the files will be extracted to and cleaned up.
    """
    # Hardcoded unwanted prefix based on the original zip structure
    unwanted_prefix = 'content/drive/MyDrive/ISIC_2019_Zipped'
    
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Unzip the files
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

    cleanup_and_move_files(target_dir, unwanted_prefix)

def cleanup_and_move_files(target_dir, unwanted_prefix):
    """
    Moves files from nested directories to the target directory and removes empty directories.
    
    Parameters:
    - target_dir: Target directory where files should ultimately reside.
    - unwanted_prefix: Prefix to identify unwanted nested directories for removal.
    """
    for root, dirs, files in os.walk(target_dir, topdown=False):
        # Move files to the target directory and remove empty directories
        for name in files:
            original_path = os.path.join(root, name)
            target_path = os.path.join(target_dir, name)
            
            if original_path != target_path:
                shutil.move(original_path, target_path)
        
        for name in dirs:
            dir_path = os.path.join(root, name)
            # Remove the directory if it's part of the unwanted structure or empty
            if unwanted_prefix in dir_path or not os.listdir(dir_path):
                shutil.rmtree(dir_path, ignore_errors=True)


Example on google colab.

In [ ]:
#| hide

# # Base directory where the datasets are stored
# dataset_dir = "/content/isic_dataset"

# # Specify the paths for the zip files
# training_zip_path = '/content/drive/MyDrive/ISIC_2019_Zipped/ISIC_2019_Training_Resized.zip'
# test_zip_path = '/content/drive/MyDrive/ISIC_2019_Zipped/ISIC_2019_Test_Resized.zip'

# # Derive the target directories for training and test datasets
# training_target_dir = os.path.join(dataset_dir, "ISIC_2019_Training_Resized")
# test_target_dir = os.path.join(dataset_dir, "ISIC_2019_Test_Resized")

# # Unzip and cleanup for both datasets
# unzip_and_cleanup(training_zip_path, training_target_dir)
# unzip_and_cleanup(test_zip_path, test_target_dir)


Main dataloader functions:

In [ ]:
#| export


def get_supervised_isic_test_dls(bs, 
                                 dataset_dir,  # Added parameter for dataset directory
                                 size=None,#preset by default to 256.
                                 device='cpu', 
                                 pct_dataset=1.0, 
                                 num_workers=12):
    max_retries = 3
    retry_delay = 1

    for attempt in range(max_retries):
        try:
            resized_dir = os.path.join(dataset_dir, "ISIC_2019_Test_Resized")
            labels_file = os.path.join(resized_dir, "labels.csv")

            with open(labels_file, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip the header row
                labels = {row[0]: row[1] for row in reader}

            resized_fnames = list(labels.keys())

            subset_size = int(len(resized_fnames) * pct_dataset)
            _resized_fnames = [os.path.join(resized_dir, fname) for fname in resized_fnames[:subset_size]]

            dls = ImageDataLoaders.from_path_func(
                resized_dir,
                _resized_fnames,
                lambda x: labels[os.path.basename(x)],
                bs=bs,
                valid_pct=0,
                device=device,
                num_workers=num_workers * (device == 'cuda'),
                drop_last=False,
            )

            return dls
        except IOError as e:
            print(f"Attempt {attempt + 1}/{max_retries} failed with IOError: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                raise

def get_supervised_isic_train_dls(bs, 
                                  dataset_dir,  # Added parameter for dataset directory
                                  size=None, #preset by default to 256.
                                  device='cpu',
                                  pct_dataset=1.0, 
                                  num_workers=12):
    max_retries = 3
    retry_delay = 1

    for attempt in range(max_retries):
        try:
            resized_dir = os.path.join(dataset_dir, "ISIC_2019_Training_Resized")
            labels_file = os.path.join(resized_dir, "labels.csv")

            with open(labels_file, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip the header row
                labels = {row[0]: row[1] for row in reader}

            resized_fnames = list(labels.keys())

            subset_size = int(len(resized_fnames) * pct_dataset)
            _resized_fnames = [os.path.join(resized_dir, fname) for fname in resized_fnames[:subset_size]]

            dls = ImageDataLoaders.from_path_func(
                resized_dir,
                _resized_fnames,
                lambda x: labels[os.path.basename(x)],
                bs=bs,
                valid_pct=0,
                device=device,
                num_workers=num_workers * (device == 'cuda'),
                drop_last=False,
            )

            return dls
        except IOError as e:
            print(f"Attempt {attempt + 1}/{max_retries} failed with IOError: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                raise


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()